In [1]:
import os
import pandas as pd
import numpy as np
import math
from multiprocessing import Pool
from itertools import combinations
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from statsmodels.tsa.statespace.sarimax import SARIMAX

def log_transf(x):
    """
    Apply a logarithmic transformation to the input x.

    Arguments
    ----------
    x : float
        the input value to be transformed.

    Returns
    ----------
    float or None
        The logarithmic transformation of the input x if it's not equal to 0 and not NaN, otherwise returns None.
    """
    if (x != 0 and x != np.nan):
        return np.log(x)
    if (x == 0 and x != np.nan):
        return 0
    if x == np.nan:
        return np.nan

In [2]:
def predict_new_deaths(window:int,train:int,p:int,q:int,ps:int,qs:int,w:int,d:int,ds:int,exo:list[str])-> pd.DataFrame:
    
    """
    This function leverages hyperparameters and exogenous variables to forecast new deaths for a given number of horizons/weeks for each window.

    Arguments
    ----------
    window : int
        window number
    train : int
        the size of training set
    p : int
        the value for auto regression order
    q : int
        the value for moving average order
    ps : int
        the value for seasonal auto regression order
    qs : int
        the value for seasonal moving average order
    w : int
        the number of time steps for a single seasonal period
    d : int
        the value for differencing 
    ds : int
        the value for seasonal differencing 
    exo : list[str]
        the list of exogenous variables
        
    Returned Values
    ----------
    forecast_f : pd.DataFrame
        DataFrame containing the predicted new deaths for each horizon.

    """
    col_core = 'new_deaths'
    df_US = pd.read_csv('OWID_weekly.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    df_US = df_US.drop(columns=['date'])
    
    #Apply log transformation
    df_US_log = pd.DataFrame()
    for col in df_US.columns:
        df_US_log[col] = df_US[col].apply(lambda x: log_transf(x))
    
    #List of exogenous variables
    total_features = ['icu_patients', 'hosp_patients','positive_rate','new_cases','new_tests' , 'people_vaccinated', 'people_fully_vaccinated']

    df_nd1 = df_US_log[col_core]
    df_variables = df_US_log[total_features]

    #Predicting weeks in future according to the horizon (h) 
    for h in range(1,5):
        
      #Shifting exogenous variables according to the horizon (h)   
      x_lagged_variables = df_variables.shift(h).bfill()
      exog_variables = x_lagged_variables[exo]
    
      #Moving window
      df_current_old14 = df_nd1[int(window-1):int(window-1+train+h)]

      i = h-1
      
      #Defining train and test data
      col_train = df_nd1.loc[df_current_old14.index][0:train]
      exo_train = exog_variables.loc[df_current_old14.index][0:train]
      exo_test = exog_variables.loc[df_current_old14.index][train:]
    
      #Model definition for prediction
      model = SARIMAX(col_train, exog=exo_train, order=(p,d,q), seasonal_order=(ps,ds,qs,w), enforce_stationarity=False, enforce_invertibility=False)
      model_fit = model.fit(disp=False)
      forecast = model_fit.predict(len(col_train), len(col_train)+i, exog = exo_test)
      
      #Inverse transformation
      forecast = np.exp(forecast)
      forecast_f = forecast.to_frame()
      
      #For each horizon, the function selectively uses values that have a lag/shift equal to the specific horizon
      if h == 1:
        df1 = forecast_f.rename(columns= {0: 'new_deaths'})
        value1 = df1['new_deaths'].iloc[i]
        index1 = df1['new_deaths'].index[i]

      elif h == 2:
        df2 = forecast_f.rename(columns= {'predicted_mean': 'new_deaths'})
        value2 = df2['new_deaths'].iloc[i]
        index2 = df2['new_deaths'].index[i]

      elif h == 3:
        df3 = forecast_f.rename(columns= {'predicted_mean': 'new_deaths'})
        value3 = df3['new_deaths'].iloc[i]
        index3 = df3['new_deaths'].index[i]

      elif h == 4:
        df4 = forecast_f.rename(columns= {'predicted_mean': 'new_deaths'})
        value4 = df4['new_deaths'].iloc[i]
        index4 = df4['new_deaths'].index[i]
        
        #Generate a DataFrame containing the predicted values
        data = {'index': [index1, index2, index3, index4],'new_deaths': [value1, value2, value3, value4]}
        forecast_f = pd.DataFrame(data=data)
        forecast_f = forecast_f.set_index('index')

    return forecast_f

In [3]:
def window_results_nd(n_train:int, n_test:int,p:int,q:int,ps:int,qs:int,w:int,d:int,ds:int,exo:list[str])-> pd.DataFrame:
     
    """
    A function that returns the weekly prediction for each window. 

    Arguments
    ----------
    n_train : int
        the size of training set
    n_test : int
        the size of testing set
    p : int
        the value for auto regression order
    q : int
        the value for moving average order
    ps : int
        the value for seasonal auto regression order
    qs : int
        the value for seasonal moving average order
    w : int
        the number of time steps for a single seasonal period
    d : int
        the value for differencing 
    ds : int
        the value for seasonal differencing 
    exo : list[str]
        the list of exogenous variables
        
    Returned Values
    ----------
    temp_results : pd.DataFrame
        A DataFrame containing weekly predictions for each window

    """  
    col_core = 'new_deaths'
    temp_results = pd.DataFrame()
    for i in range(n_test):
        window = i + 1
        predict_current = predict_new_deaths(window,n_train,p,q,ps,qs,w,d,ds,exo)
        predict_current = predict_current.rename(columns={str(col_core):"window_"+str(window)})
        temp_results = pd.concat([temp_results, predict_current], axis=1)
    return temp_results

In [4]:
def output_h_nd(temp_results: pd.DataFrame, h: int, n_test:int)-> pd.DataFrame:
    """
    This function processes the 'temp_results' DataFrame, which contains predictions for h weeks ahead for each window, and organizes the data into a single column for each horizon.

    Arguments
    ----------
    temp_results : pd.DataFrame
        the dataframe that has h weeks ahead prediction for each window
    h : int
        horizon value
    n_test : int
        the size of testing set

    Returned Values
    ----------
    df_results_python : pd.DataFrame
        Dataframe for each horizon.  

    """ 
    n_iter = n_test-h+1
    df_results = []
    if h==1:
        for i in range(int(n_iter)):
          window = i + 1
          current_results = temp_results[['window_'+str(window)]].dropna()
          current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
          df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())
    if h>1:
        for i in range(int(h-1)):
          window = i + 1
          current_results = temp_results[['window_'+str(window)]].dropna()
          current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
          df_results.append(pd.DataFrame(current_results.iloc[0]).transpose())

        for i in range(int(n_iter)):
          window = i + 1
          current_results = temp_results[['window_'+str(window)]].dropna()
          current_results = current_results.rename(columns={'window_'+str(window): "h="+str(h)})
          df_results.append(pd.DataFrame(current_results.iloc[int(h-1)]).transpose())
    df_results_python = pd.concat(df_results,axis=0)
    return df_results_python

In [5]:
def predict_table_nd(n_test, temp_results)-> pd.DataFrame:
    """
    A function that iterates through each horizon and combines the predictions for each horizon into a final DataFrame that has 'h' columns for each week.

    Arguments
    ----------
    n_test : int
        the size of testing set
    temp_results : pd.DataFrame
        the dataframe that has h weeks ahead prediction for each window

    Returned Values
    ----------
    df_final_forecast : pd.DataFrame
        DataFrame presenting the h weeks ahead prediction for each week.

    """ 
    df_final_forecast = pd.DataFrame()
    for h in range(1,5):
        df_current = output_h_nd(temp_results, h, n_test)
        df_final_forecast = pd.concat([df_final_forecast, df_current], axis=1)
    return df_final_forecast

In [6]:
def smape(y_true, y_pred):
    """ 
    A function to calculate the symmetric mean absolute percentage error (sMAPE).

    Arguments
    ----------
    y_true: pd.DataFrame
        the original data
    y_pred: pd.DataFrame
        the predicted data

    Returned Values
    ----------
    smape : float
        returns sMAPE scores

    """  
    smape = (+1/len(y_true) * np.sum(2 * np.abs(y_pred-y_true) / (np.abs(y_true) + np.abs(y_pred))*100))
    return round(smape,2)

In [7]:
def smape_results_nd(df_final_forecast, p):
    """ 
    This function calculates the sMAPE score using the sMAPE definition between the original data and the predictions for each horizon
    
    Arguments
    ----------
    df_final_forecast: pd.DataFrame
        the predicted DataFrame containing h weeks ahead prediction for each week
    p : int
        the value for auto regression order

    Returned Values
    ----------
    df_results : pd.DataFrame
        DataFrame with SMAPE scores for each horizon and the average SMAPE score.
        
    """  
    col_core = 'new_deaths'
    df_US = pd.read_csv('OWID_weekly.csv')
    df_US.index = pd.to_datetime(df_US['date'])
    h_level = df_final_forecast.columns.to_list()
    eval = []
    for cols in h_level:
        y_true = df_US.loc[df_final_forecast.index][col_core]
        y_pred = df_final_forecast[cols]
        value = smape(y_true, y_pred)
        eval.append(value)
    eval.append(np.mean(eval))
    df_results = pd.DataFrame(eval, columns=[str(col_core)+"_p="+str(p)])
    h_level.append('Average')
    df_results.index = h_level
    return df_results

In [8]:
#For reproducibility
import warnings
warnings.filterwarnings('ignore')
n_train = 40
n_test = 72
col_core = 'new_deaths'
w = 10
p = 1
d = 1
q = 0
ps = 1
ds = 0
qs = 0
exo_list = [['icu_patients','positive_rate']]
for exo in exo_list:
        temp_results = window_results_nd(n_train, n_test,p,q,ps,qs,w,d,ds,exo)
        df_final_forecast = predict_table_nd(n_test, temp_results)
        df_smape_results = smape_results_nd(df_final_forecast, p)
        print("sMAPE {}".format(df_smape_results))

sMAPE          new_deaths_p=1
h=1             10.2900
h=2             10.3600
h=3             14.0400
h=4             21.0200
Average         13.9275


In [9]:
# import warnings
# warnings.filterwarnings('ignore')
# n_train = 40
# n_test = 72
# col_core = 'new_deaths'
# w = 10
# p = 1
# d = 1
# q = 0
# ps = 1
# ds = 0
# qs = 0
# all_features = ['icu_patients', 'hosp_patients', 'positive_rate','new_cases','new_tests','people_vaccinated', 'people_fully_vaccinated']
# for r in range(1,len(all_features)+1):
#     new_feature_list = list(combinations(all_features,r))
#     for items in new_feature_list:
#         exo = list(items)
#         print("Exogenous features:" , exo)
#         temp_results = window_results_nd(n_train, n_test,p,q,ps,qs,w,d,ds,exo)
#         df_final_forecast = predict_table_nd(n_test, temp_results)
#         df_smape_results = smape_results_nd(df_final_forecast, p)
#         print("sMAPE {}".format(df_smape_results))